In [1]:
import jieba  
import jieba.posseg as pseg  
import os  
import sys  
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer 
import numpy as np
import pickle
from scipy.sparse import coo_matrix, vstack
from data_learning import Transform
from brand_obj import Brand
from data_tool import flat_json,csv_to_list, list_to_csv
from collections import OrderedDict, Counter
from async_promises import Promise


number of posts
number of class 1 post
number of class 2 post
number of class 3 post
start fans 增長率
end fans

In [2]:
fp_profile = csv_to_list("fid_list_4.csv")

In [3]:
def make_promise(fp):
    # 做spss 資料，如果成功
    def resolver(resolve, reject):
        fp_dict = OrderedDict()
        try:
            fp_dict['fid'] = fp['fid']
            fp_dict['name'] = fp['name']
            fp_dict['category'] = fp['category']
            fp_dict['fb_certificate'] = fp['fb_certificate']   #0:no certificate, 1:grey certificate, 2:blue certificate
        except:
            print "fp problem : {0}".format(fp['fid'])
            resolve(fp_dict)
        
        try:
            brand = Brand(str(fp['fid']))
            fp_dict['like_list'] = brand.like_count_list
            fp_dict['comment_list'] = brand.comment_count_list
            fp_dict['share_list'] = brand.share_count_list
            fp_dict['friendtag_list'] = brand.tag_count_list
        except:
            print "Brand count problem : {0}".format(fp['fid'])
            resolve(fp_dict)
        
        try:
            fp_dict['like_avg_count'] = sum(fp_dict['like_list'])/float(len(fp_dict['like_list']))
            fp_dict['comment_avg_count'] = sum(fp_dict['comment_list'])/float(len(fp_dict['comment_list']))
            fp_dict['share_avg_count'] = sum(fp_dict['share_list'])/float(len(fp_dict['share_list']))
            fp_dict['friendtag_avg_count'] = sum(fp_dict['friendtag_list'])/float(len(fp_dict['friendtag_list']))
        except:
            print "Brand count math problem : {0}".format(fp['fid'])
            resolve(fp_dict)
        
        try:
            fp_dict['class_list'] = brand.class_list
            fp_dict['class1_post_count'] = fp_dict['class_list'].count(1)
            fp_dict['class2_post_count'] = fp_dict['class_list'].count(2)
            fp_dict['class3_post_count'] = fp_dict['class_list'].count(3)
#             class_dict = Counter(fp_dict['class_list'])
#             [fp_dict['class1_post_count'],
#             fp_dict['class2_post_count'],
#             fp_dict['class3_post_count']] = [v for k,v in class_dict.iteritems()]
        except:
            print "class count problem : {0}".format(fp['fid'])
            resolve(fp_dict)

        fp_dict['start_fans'] = brand.start_fan_count
        fp_dict['stop_fans'] = brand.stop_fan_count
        resolve(fp_dict)
            
    return Promise(resolver)

In [4]:
# fp_spss_list = []
promise_list = []
for fp in fp_profile:
# for fp in [fp_profile[164]]:
    p = make_promise(fp)
    promise_list.append(p)

In [5]:
p = Promise.all(promise_list).then(lambda res: res)
result = p.get()

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/q1/sy8zj9z12v792k86g84xby1r0000gn/T/jieba.cache
Loading model cost 1.377 seconds.
Prefix dict has been built succesfully.


In [6]:
len(result)

230

In [7]:
list_to_csv(result,"data_spss/spss_0922.csv")